# SQL Injection Detection Experiment
Using SQLiV3.csv (baseline) and SQLiV5.json (adversarial)
---

In [17]:
# Step 1: Install Required Libraries
!pip install pandas scikit-learn tabulate

In [18]:
# Step 2: Import Libraries
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from tabulate import tabulate

In [19]:
# Step 3: Load Baseline Dataset
baseline_df = pd.read_csv('SQLiV3.csv')
baseline_df.columns = ['pattern', 'type']
baseline_df['label'] = baseline_df['type'].apply(lambda x: 1 if str(x).strip() == '1' else 0)
baseline_df.head()

,pattern,type,label
0,""" or pg_sleep ( __TIME__ ) --",1,1
1,create user name identified by pass123 tempora...,1,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1,1
3,select * from users where id = '1' or @ @1 ...,1,1
4,"select * from users where id = 1 or 1#"" ( ...",1,1


In [20]:
# Step 4: Train/Test Split
# Handle missing values by dropping rows where 'pattern' is NaN
baseline_df.dropna(subset=['pattern'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    baseline_df['pattern'], baseline_df['label'], test_size=0.2, random_state=42
)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [21]:
# Step 5: Train Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000)

In [22]:
# Step 6: Evaluate on Clean Test Set
y_pred_clean = model.predict(X_test_vec)
report_clean = classification_report(y_test, y_pred_clean, output_dict=True)
df_clean_report = pd.DataFrame(report_clean).transpose()
print('Clean Test Set Results:')
print(tabulate(df_clean_report, headers='keys', tablefmt='grid', showindex=True))

Clean Test Set Results:
+--------------+-------------+----------+------------+-------------+
|              |   precision |   recall |   f1-score |     support |
+==============+=============+==========+============+=============+
| 0            |    0.966998 | 0.994133 |   0.980377 | 3920        |
+--------------+-------------+----------+------------+-------------+
| 1            |    0.989307 | 0.941176 |   0.964642 | 2261        |
+--------------+-------------+----------+------------+-------------+
| accuracy     |    0.974761 | 0.974761 |   0.974761 |    0.974761 |
+--------------+-------------+----------+------------+-------------+
| macro avg    |    0.978152 | 0.967655 |   0.97251  | 6181        |
+--------------+-------------+----------+------------+-------------+
| weighted avg |    0.975158 | 0.974761 |   0.974621 | 6181        |
+--------------+-------------+----------+------------+-------------+


In [23]:
# Step 7: Load Adversarial Dataset
with open('SQLiV5.json', 'r') as f:
    adv_data = json.load(f)
adv_df = pd.DataFrame(adv_data)
adv_df.columns = ['pattern', 'type']
adv_df['label'] = adv_df['type'].apply(lambda x: 1 if x.strip().lower() == 'sqli' else 0)
X_adv_vec = vectorizer.transform(adv_df['pattern'])
y_adv_pred = model.predict(X_adv_vec)
report_adv = classification_report(adv_df['label'], y_adv_pred, output_dict=True)
df_adv_report = pd.DataFrame(report_adv).transpose()
print('Adversarial Test Set Results:')
print(tabulate(df_adv_report, headers='keys', tablefmt='grid', showindex=True))

Adversarial Test Set Results:
+--------------+-------------+----------+------------+--------------+
|              |   precision |   recall |   f1-score |      support |
+==============+=============+==========+============+==============+
| 0            |    0.951436 | 0.997797 |   0.974065 | 19517        |
+--------------+-------------+----------+------------+--------------+
| 1            |    0.997857 | 0.952696 |   0.974753 | 21013        |
+--------------+-------------+----------+------------+--------------+
| accuracy     |    0.974414 | 0.974414 |   0.974414 |     0.974414 |
+--------------+-------------+----------+------------+--------------+
| macro avg    |    0.974647 | 0.975246 |   0.974409 | 40530        |
+--------------+-------------+----------+------------+--------------+
| weighted avg |    0.975503 | 0.974414 |   0.974422 | 40530        |
+--------------+-------------+----------+------------+--------------+


In [24]:
# Step 8: Combined Results Table
metrics = ['precision', 'recall', 'f1-score']
combined_df = pd.DataFrame({
    'Metric': metrics,
    'Clean Dataset': [report_clean['weighted avg'][m] for m in metrics],
    'Adversarial Dataset': [report_adv['weighted avg'][m] for m in metrics]
})
print('Combined Performance Comparison:')
print(tabulate(combined_df, headers='keys', tablefmt='fancy_grid', showindex=False))

Combined Performance Comparison:
╒═══════════╤═════════════════╤═══════════════════════╕
│ Metric    │   Clean Dataset │   Adversarial Dataset │
╞═══════════╪═════════════════╪═══════════════════════╡
│ precision │        0.975158 │              0.975503 │
├───────────┼─────────────────┼───────────────────────┤
│ recall    │        0.974761 │              0.974414 │
├───────────┼─────────────────┼───────────────────────┤
│ f1-score  │        0.974621 │              0.974422 │
╘═══════════╧═════════════════╧═══════════════════════╛
